In [10]:
# section 1 load all the necessary modules and packages
import glob
import time
import geopandas as gpd
import netCDF4 as nc4
import numpy as np
import pandas as pd
import xarray as xr
from shapely.geometry import Polygon
# not neccessary for the function but for visualziation
import matplotlib.pyplot as plt
import warnings
import sys
#from   easymore.easymore import easymore
import os


In [11]:
# define the target river segment
target_segment = 71021602 # segment that passes Saskatoon

# netwrok topology with lakes

In [12]:
cat = gpd.read_file('../data/shp/cat_lake_area_corrected_'+str(target_segment)+'_merit.shp')
riv = gpd.read_file('../data/shp/riv_lake_length_corrected_'+str(target_segment)+'_merit.shp')
resolved_lakes = gpd.read_file('../data/shp/resolved_lake_'+str(target_segment)+'_merit.shp')
riv_lake = gpd.read_file('../data/shp/riv_lake_'+str(target_segment)+'_merit.shp')

IDs = np.unique(np.append(riv.seg_id,resolved_lakes.lake_id)) # merge the ID of segments, and lakes
data = {'ID': IDs}
NTOPO = pd.DataFrame(data=data)

# creaton of other field for NTOPO
NTOPO ['ID_next'] = -9999
NTOPO ['area'] = 1 # m2
NTOPO ['length'] = 1 # m
NTOPO ['is_lake'] = None
NTOPO ['outflow'] = None
NTOPO ['inflow'] = None
NTOPO ['endorheic'] = 0 # 
NTOPO ['slope'] = 0.001 # default value but can be read from riv
NTOPO ['lake_vol'] = -9999


warnings.simplefilter('ignore') # silent the warning
for index, row in NTOPO.iterrows():
    
    if row.ID in np.array(riv['seg_id']): # the ID is a river segment
        
        # slice the riv and allocate the values
        riv_slice = riv[riv['seg_id']==row.ID]
        
        # is_lake set to zero, length and inflow, outflow of possible lakes
        NTOPO ['is_lake'].loc[index] = 0 # object is not a lake
        NTOPO ['length'].loc[index]  = riv_slice['length'].iloc[0] * riv_slice['len_ratio'].iloc[0]
        NTOPO ['outflow'].loc[index] = riv_slice['outflow'].iloc[0]
        NTOPO ['inflow'].loc[index]  = riv_slice['inflow'].iloc[0]
        NTOPO ['ID_next'].loc[index] = riv_slice['to_seg_id'].iloc[0]
        
        # check if it intersect with any lake
        if (riv_slice['outflow'].iloc[0]<0) or (riv_slice['inflow'].iloc[0]<0): # do no intersect with lake
            NTOPO ['ID_next'].loc[index] = riv_slice['to_seg_id'].iloc[0]
        else: # intersect with lakes
            # get the lake ID
            temp = riv_lake[riv_lake['seg_id'] == riv_slice['seg_id'].iloc[0]]
            lake_ID = temp['lake_id'].iloc[0]
            # check the output and input
            if riv_slice['inflow'].iloc[0] == 1: # inflow to the lake, update the ID_next
                NTOPO ['ID_next'].loc[index] = lake_ID
            if riv_slice['outflow'].iloc[0] == 1: # make sure the upstream segment of outflow are draing to lake
                riv_up = riv[riv['to_seg_id'] == riv_slice['seg_id'].iloc[0]] # find upstream
                if not riv_up.empty:
                    up_IDs = np.array(riv_up['seg_id'])
                    temp = NTOPO [NTOPO['ID'].isin(up_IDs)]
                    indx = temp.index
                    NTOPO['ID_next'].loc[indx] = lake_ID
                    # print('check lake:', lake_ID)
        
        # slice the cat to the target id
        cat_slice = cat[cat['cat_id']==row.ID]
        if not cat_slice.empty:
            NTOPO ['area'].loc[index] = cat_slice['cat_area'].iloc[0] * cat_slice['area_ratio'].iloc[0]
        
    if row.ID in np.array(resolved_lakes['lake_id']): # the ID is a lake
        
        resolved_lakes_slice = resolved_lakes[resolved_lakes['lake_id']==row.ID]
        
        # is_lake set to zero
        NTOPO ['is_lake'].loc[index] = 1 # object is a lake
        NTOPO ['area'].loc[index] = resolved_lakes_slice['lake_area'].iloc[0] # object is a lake
        NTOPO ['outflow'].loc[index] = -1
        NTOPO ['inflow'].loc[index] = -1
        NTOPO ['lake_vol'].loc[index] = resolved_lakes_slice['lake_vol'].iloc[0] # object is a lake
        
        # slice the riv_lake to the target id
        riv_lake_slice = riv_lake[riv_lake['lake_id']==row.ID]
        riv_lake_slice = riv_lake_slice[riv_lake_slice['outflow']==1] # get the outflow
        if not riv_lake_slice.empty: # if outfow existis (not endirehic lakes)
            NTOPO ['ID_next'].loc[index] = riv_lake_slice['seg_id'].iloc[0]
        else: # the lake doest not any outflow and is set to endorheic
            NTOPO ['endorheic'] = 1

warnings.simplefilter('default') # silent the warning

NTOPO
NTOPO.to_csv('../data/network_topology/Network_topology_merit_with_lake.csv')



## match the grand id and lake id for the lake_param_HYPE_slice and save in lake_param_HYPE_grand_to_HydroLake

In [13]:
df = pd.read_csv('../data/lake_param/lake_param_HYPE.csv')
link = pd.read_csv('../data/lake_param/lake_param_HYPE_grand_to_HydroLake.csv')


df = pd.merge(link,df, on=['GRAND_ID'], how='inner')
df.to_csv('../data/lake_param/lake_param_HYPE_Saskatoon.csv')



In [14]:
Ntopo = pd.read_csv('../data/network_topology/Network_topology_merit_with_lake.csv')
Hanasaki = pd.read_csv('../data/lake_param/lake_param_Hanasaki_Saskatoon.csv')
HYPE = pd.read_csv('../data/lake_param/lake_param_HYPE_Saskatoon.csv')

Ntopo_before = len(Ntopo)

Hanasaki['ID'] = Hanasaki['lakeId']
HYPE['ID'] = HYPE['Hylak_id_x']

HYPE_before = print(len(HYPE))

# if there is a HYPE and Hanasaki choose Hanasaki
shared_ID = np.intersect1d(np.array(Hanasaki['ID']), np.array(HYPE['ID']))
if shared_ID.size == 0:
    HYPE_retain = np.setdiff1d(np.array(HYPE['ID']), shared_ID)
    HYPE = HYPE [HYPE['ID'].isin(HYPE_retain)]

HYPE_after = print(len(HYPE))

if HYPE_before != HYPE_after:
    print('the IDs were removed from the HYPE, to be considered as Hanasaki')
    print(shared_ID)
    if len(shared_ID) == (HYPE_before - HYPE_after):
        print('number of HYPE reduced:', HYPE_before - HYPE_after)
        print('number of shared_ID', len(shared_ID))
    else:
        print('sth is wrong!')


Ntopo = pd.merge(Ntopo, Hanasaki, on=['ID'], how='outer')
Ntopo = pd.merge(Ntopo, HYPE, on=['ID'], how='outer')


Ntopo_after = len(Ntopo)

Ntopo = Ntopo.fillna(-9999)

Ntopo ['lake_type_Doll'] = Ntopo ['is_lake'] # in case all the lakes area

Ntopo ['lake_type'] = Ntopo ['is_lake'] # set all lakes as Doll

Ntopo ['PRM_FLAG'] = 0

Ntopo ['H06_I_mem_F'] = 1
Ntopo ['H06_D_mem_F'] = 1
Ntopo ['H06_I_mem_L'] = 5
Ntopo ['H06_D_mem_L'] = 5

Ntopo ['Doll_Coef']  = 0.01 
Ntopo ['Doll_Power'] = 1.5
Ntopo ['Doll_MaxStorage'] = Ntopo ['area'] * 5 + Ntopo ['lake_vol']


indx = Ntopo.index [Ntopo['length']<1]
Ntopo['length'].loc[indx] = 1.00

# add lake type, first HYPE and then HANASAKI
indx = Ntopo.index[Ntopo['ID'].isin(np.array(HYPE['ID']))]
print(indx)
Ntopo ['lake_type'].loc[indx] = 3 # HYPE
Ntopo ['PRM_FLAG'].loc[indx] = 1 # flag
Ntopo ['lake_type_Doll_HYPE'] = Ntopo ['lake_type'] # mix of HYPE and Doll
# add lake type, first HYPE and then HANASAKI
indx = Ntopo.index[Ntopo['ID'].isin(np.array(Hanasaki['ID']))]
print(indx)
Ntopo ['lake_type'].loc[indx] = 2 # Hanasaki


Ntopo ['lake_type_Doll_HYPE_HANASAKI'] = Ntopo ['lake_type']

if Ntopo_before == Ntopo_after:
    print('Ntopo before and after are the same length, seems good to go!')
    Ntopo.to_csv('../data/network_topology/Network_topology_merit_with_lake.csv')
else:
    print('sth is wrong, Ntopo before and after are different')



11
11
Int64Index([0, 4, 5, 6, 9, 10, 13, 14, 15, 16, 17], dtype='int64')
Int64Index([0], dtype='int64')
Ntopo before and after are the same length, seems good to go!


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [15]:
# save the network topology
df = pd.read_csv('../data/network_topology/Network_topology_merit_with_lake.csv')

# prepare and rename some variables for Doll 2003
df['D03_S_0'] = df['lake_vol'] # create the new variable
# rename to more understandbale variables
new_names_Doll = {'D03_S_0': 'D03_S_0',
                  'Doll_Coef': 'D03_Coef',
                  'Doll_Power': 'D03_Power',
                  'Doll_MaxStorage': 'D03_S_max'}
# define the variable name, long name and units
var_info_Doll = {'D03_S_0':   {'long_name': 'lake inactive volume',  'unit': 'm**3'},
                 'D03_Coef':  {'long_name': 'Doll coefficient often taken as 0.01 per day', 'unit': 'd*-1'},
                 'D03_Power': {'long_name': 'Doll power',  'unit': '-'},
                 'D03_S_max': {'long_name': 'lake maximume storage, often 5 meter above inactive storage', 'unit': 'm**3'}}
# define the saving mechanism
var_encoding_Doll = {'D03_S_0':   {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                     'D03_Coef':  {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                     'D03_Power': {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                     'D03_S_max': {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9}}
#
new_names_Doll_values = list(new_names_Doll.values())
var_info_Doll_keys = list(var_info_Doll.keys())
var_encoding_Doll_keys = list(var_encoding_Doll.keys())
if not new_names_Doll_values == var_info_Doll_keys == var_encoding_Doll_keys:
    sys.exit('keys and values are not similar')


# prepare and rename varibales for Hanasaki 2006
new_names_Hanasaki = {'H06_alpha': 'H06_alpha',
                      'H06_envfact': 'H06_envfact',
                      'H06_c1': 'H06_c1',
                      'H06_c2':'H06_c2',
                      'H06_exponent':'H06_exponent',
                      'H06_denominator': 'H06_denominator',
                      'H06_c_compare': 'H06_c_compare',
                      'H06_frac_Sdead': 'H06_frac_Sdead',
                      'H06_I_mem_F': 'H06_I_mem_F',
                      'H06_D_mem_F': 'H06_D_mem_F',
                      'H06_I_mem_L': 'H06_I_mem_L',
                      'H06_D_mem_L': 'H06_D_mem_L',
                      'H06_E_rel': 'H06_E_rel_init',
                      'H06_purpose': 'H06_purpose',
                      'inflow_Jan': 'H06_I_Jan',
                      'inflow_Feb': 'H06_I_Feb',
                      'inflow_Mar': 'H06_I_Mar',
                      'inflow_Apr': 'H06_I_Apr',
                      'inflow_May': 'H06_I_May',
                      'inflow_Jun': 'H06_I_Jun',
                      'inflow_Jul': 'H06_I_Jul',
                      'inflow_Aug': 'H06_I_Aug',
                      'inflow_Sep': 'H06_I_Sep',
                      'inflow_Oct': 'H06_I_Oct',
                      'inflow_Nov': 'H06_I_Nov',
                      'inflow_Dec': 'H06_I_Dec',
                      'H06_D_Jan': 'H06_D_Jan',
                      'H06_D_Feb': 'H06_D_Feb',
                      'H06_D_Mar': 'H06_D_Mar',
                      'H06_D_Apr': 'H06_D_Apr',
                      'H06_D_May': 'H06_D_May',
                      'H06_D_Jun': 'H06_D_Jun',
                      'H06_D_Jul': 'H06_D_Jul',
                      'H06_D_Aug': 'H06_D_Aug',
                      'H06_D_Sep': 'H06_D_Sep',
                      'H06_D_Oct': 'H06_D_Oct',
                      'H06_D_Nov': 'H06_D_Nov',
                      'H06_D_Dec': 'H06_D_Dec'}
# define the variable name, long name and units
var_info_Hanasaki =  {'H06_alpha':       {'long_name': 'fraction of active storage compared to the total storage',  'unit': '-'},
                      'H06_envfact':     {'long_name': 'fraction of yearly mean inflow that can be used to meet demands',  'unit': '-'},
                      'H06_c1':          {'long_name': 'first coefficient of target release calculation',  'unit': '-'},
                      'H06_c2':          {'long_name': 'second coefficient of target release calculation',  'unit': '-'},
                      'H06_exponent':    {'long_name': 'exponent in actual release calculation',  'unit': 'm**3'},
                      'H06_denominator': {'long_name': 'denominator in actual release calculation',  'unit': 'm**3'},
                      'H06_c_compare':   {'long_name': 'threshold to compare within year or multiyear reservoirs, often fixed at 0.5',  'unit': '-'},
                      'H06_frac_Sdead':  {'long_name': 'fraction of maximume storage that define dead storage, 0.1',  'unit': '-'},
                      'H06_I_mem_F':     {'long_name': 'logical parameter to activate memory for inflow',  'unit': '-'},
                      'H06_D_mem_F':     {'long_name': 'logical parameter to activate memory for demand',  'unit': '-'},
                      'H06_I_mem_L':     {'long_name': 'the length of the memory in years for inflow',  'unit': '-'},
                      'H06_D_mem_L':     {'long_name': 'the length of the memory in years for demand',  'unit': '-'},
                      'H06_E_rel_init':  {'long_name': 'release before the hanasaki start year',  'unit': '-'},
                      'H06_purpose':     {'long_name': 'Reservoir purpose (0 non-irrigation, 1 irrigation)',  'unit': '-'},
                      'H06_I_Jan':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_Feb':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_Mar':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_Apr':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_May':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_Jun':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_Jul':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_Aug':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_Sep':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_Oct':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_Nov':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_I_Dec':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Jan':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Feb':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Mar':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Apr':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_May':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Jun':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Jul':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Aug':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Sep':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Oct':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Nov':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'},
                      'H06_D_Dec':       {'long_name': 'lake inactive volume',  'unit': 'm**3 s**-1'}}
# define saving mechanism
var_encoding_Hanasaki ={'H06_alpha':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_envfact':     {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_c1':          {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_c2':          {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_exponent':    {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_denominator': {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_c_compare':   {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_frac_Sdead':  {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_mem_F':     {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                        'H06_D_mem_F':     {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                        'H06_I_mem_L':     {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                        'H06_D_mem_L':     {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                        'H06_E_rel_init':  {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_purpose':     {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                        'H06_I_Jan':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_Feb':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_Mar':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_Apr':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_May':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_Jun':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_Jul':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_Aug':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_Sep':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_Oct':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_Nov':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_I_Dec':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Jan':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Feb':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Mar':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Apr':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_May':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Jun':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Jul':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Aug':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Sep':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Oct':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Nov':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                        'H06_D_Dec':       {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},}
#
new_names_Hanasaki_values = list(new_names_Hanasaki.values())
var_info_Hanasaki_keys = list(var_info_Hanasaki.keys())
var_encoding_Hanasaki_keys = list(var_encoding_Hanasaki.keys())
if not new_names_Hanasaki_values == var_info_Hanasaki_keys == var_encoding_Hanasaki_keys:
    sys.exit('keys and values are not similar')


# prepare and rename HYPE parameters
new_names_HYPE   = {'EL_SPL':    'HYPE_SPL_EL',
                    'SPL_COEFF': 'HYPE_SPL_COEFF',
                    'SPL_EXP':   'HYPE_SPL_EXP',
                    'EL_LIM':    'HYPE_LIM_EL',
                    'EL_MIN':    'HYPE_MIN_EL',
                    'EL_ZERO':   'HYPE_ZERO_EL',
                    'OPS_AMP':   'HYPE_PRIM_AMP',
                    'OPS_PHA':   'HYPE_PRIM_PHASE',
                    'OPS_MEAN':  'HYPE_PRIM_MEAN',
                    'PRM_FLAG':  'HYPE_PRIM_FLAG'}
# define the varibale name, long name
var_info_HYPE = {'HYPE_SPL_EL':     {'long_name': 'fraction of active storage compared to the total storage',  'unit': '-'},
                 'HYPE_SPL_COEFF':  {'long_name': 'fraction of yearly mean inflow that can be used to meet demands',  'unit': '-'},
                 'HYPE_SPL_EXP':    {'long_name': 'first coefficient of target release calculation',  'unit': '-'},
                 'HYPE_LIM_EL':     {'long_name': 'second coefficient of target release calculation',  'unit': '-'},
                 'HYPE_MIN_EL':     {'long_name': 'exponent in actual release calculation',  'unit': 'm**3'},
                 'HYPE_ZERO_EL':    {'long_name': 'denominator in actual release calculation',  'unit': 'm**3'},
                 'HYPE_PRIM_AMP':   {'long_name': 'threshold to compare within year or multiyear reservoirs, often fixed at 0.5',  'unit': '-'},
                 'HYPE_PRIM_PHASE': {'long_name': 'threshold to compare within year or multiyear reservoirs, often fixed at 0.5',  'unit': '-'},
                 'HYPE_PRIM_MEAN':  {'long_name': 'threshold to compare within year or multiyear reservoirs, often fixed at 0.5',  'unit': '-'},
                 'HYPE_PRIM_FLAG':  {'long_name': 'threshold to compare within year or multiyear reservoirs, often fixed at 0.5',  'unit': '-'}}
# define the saving varibales
var_encoding_HYPE     = {'HYPE_SPL_EL':     {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                         'HYPE_SPL_COEFF':  {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                         'HYPE_SPL_EXP':    {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                         'HYPE_LIM_EL':     {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                         'HYPE_MIN_EL':     {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                         'HYPE_ZERO_EL':    {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                         'HYPE_PRIM_AMP':   {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                         'HYPE_PRIM_PHASE': {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                         'HYPE_PRIM_MEAN':  {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                         'HYPE_PRIM_FLAG':  {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9}}
#
new_names_HYPE_values = list(new_names_HYPE.values())
var_info_HYPE_keys = list(var_info_HYPE.keys())
var_encoding_HYPE_keys = list(var_encoding_HYPE.keys())
if not new_names_HYPE_values == var_info_HYPE_keys == var_encoding_HYPE_keys:
    sys.exit('keys and values are not similar')


# network topology
new_names_ntopo = {'ID':        'ID',
                   'ID_next':   'ID_next',
                   'length':    'length',
                   'area':      'area',
                   'slope':     'slope',
                   'is_lake':   'is_lake',
                   'lake_vol':  'lake_vol',
                   'lake_type_Doll': 'lake_type_Doll',
                   'lake_type_Doll_HYPE': 'lake_type_Doll_HYPE',
                   'lake_type_Doll_HYPE_HANASAKI': 'lake_type_Doll_HYPE_HANASAKI'}

var_info_ntopo = {'ID':        {'long_name': 'ID',        'unit': '-'},
                  'ID_next':   {'long_name': 'ID next',   'unit': '-'},
                  'length':    {'long_name': 'length',    'unit': 'm'},
                  'area':      {'long_name': 'area',      'unit': 'm**2'},
                  'slope':     {'long_name': 'slope',     'unit': 'm m**-1'},
                  'is_lake':   {'long_name': 'lake flag',     'unit': '-'},
                  'lake_vol':  {'long_name': 'lake volume',   'unit': 'm**3'},
                  'lake_type_Doll':  {'long_name': 'lake type only Doll', 'unit': '-'},
                  'lake_type_Doll_HYPE':  {'long_name': 'lake type Doll and HYPE', 'unit': '-'},
                  'lake_type_Doll_HYPE_HANASAKI': {'long_name': 'lake type Doll and HYPE and HANASAKI', 'unit': '-'}}

var_encoding_ntopo = {'ID':        {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                      'ID_next':   {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                      'length':    {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                      'area':      {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                      'slope':     {'dtype': 'float64', '_FillValue': -9999.0, 'zlib': True, 'complevel': 9},
                      'is_lake':   {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                      'lake_vol':  {'dtype': 'float64', '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                      'lake_type_Doll':  {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                      'lake_type_Doll_HYPE':  {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9},
                      'lake_type_Doll_HYPE_HANASAKI': {'dtype': 'int64',   '_FillValue': -9999  , 'zlib': True, 'complevel': 9}}
#
new_names_ntopo_values = list(new_names_ntopo.values())
var_info_ntopo_keys = list(var_info_ntopo.keys())
var_encoding_ntopo_keys = list(var_encoding_ntopo.keys())
if not new_names_ntopo_values == var_info_ntopo_keys == var_encoding_ntopo_keys:
    sys.exit('keys and values are not similar')


##
new_names = {**new_names_HYPE, **new_names_Hanasaki, **new_names_Doll, **new_names_ntopo}
var_info = {**var_info_HYPE, **var_info_Hanasaki, **var_info_Doll, **var_info_ntopo}
var_encoding = {**var_encoding_HYPE, **var_encoding_Hanasaki, **var_encoding_Doll, **var_encoding_ntopo}


print(new_names)
print(var_encoding)
print(var_info)


df.rename(columns=new_names, inplace=True)

temp = list(new_names.values())
df = df[temp] # subset to the existing column names

print(df)

df = df.set_index('ID')

ds = df.to_xarray()

ds.attrs['Conventions'] = 'CF-1.6'
ds.attrs['License']     = 'The data were written by Shervan Gharari. They are under GPL.'
ds.attrs['history']     = 'Created ' + time.ctime(time.time())
ds.attrs['source']      = 'Written by test script of utilities (https://github.com/ShervanGharari/utility-codes'


for key in var_info.keys():
    for key1 in var_info[key].keys():
        print(var_info[key][key1])
        ds[key].attrs[key1] = var_info[key][key1]
        print(ds[key].attrs[key1])

if os.path.isfile('../data/network_topology/Network_topology_merit_lake.nc'):
    os.remove('../data/network_topology/Network_topology_merit_lake.nc')

ds = ds.rename_dims({'ID': 'n'})
ds.to_netcdf('../data/network_topology/Network_topology_merit_lake.nc',\
             encoding = var_encoding)

#
ds = xr.open_dataset('../data/network_topology/Network_topology_merit_lake.nc')

ds



{'EL_SPL': 'HYPE_SPL_EL', 'SPL_COEFF': 'HYPE_SPL_COEFF', 'SPL_EXP': 'HYPE_SPL_EXP', 'EL_LIM': 'HYPE_LIM_EL', 'EL_MIN': 'HYPE_MIN_EL', 'EL_ZERO': 'HYPE_ZERO_EL', 'OPS_AMP': 'HYPE_PRIM_AMP', 'OPS_PHA': 'HYPE_PRIM_PHASE', 'OPS_MEAN': 'HYPE_PRIM_MEAN', 'PRM_FLAG': 'HYPE_PRIM_FLAG', 'H06_alpha': 'H06_alpha', 'H06_envfact': 'H06_envfact', 'H06_c1': 'H06_c1', 'H06_c2': 'H06_c2', 'H06_exponent': 'H06_exponent', 'H06_denominator': 'H06_denominator', 'H06_c_compare': 'H06_c_compare', 'H06_frac_Sdead': 'H06_frac_Sdead', 'H06_I_mem_F': 'H06_I_mem_F', 'H06_D_mem_F': 'H06_D_mem_F', 'H06_I_mem_L': 'H06_I_mem_L', 'H06_D_mem_L': 'H06_D_mem_L', 'H06_E_rel': 'H06_E_rel_init', 'H06_purpose': 'H06_purpose', 'inflow_Jan': 'H06_I_Jan', 'inflow_Feb': 'H06_I_Feb', 'inflow_Mar': 'H06_I_Mar', 'inflow_Apr': 'H06_I_Apr', 'inflow_May': 'H06_I_May', 'inflow_Jun': 'H06_I_Jun', 'inflow_Jul': 'H06_I_Jul', 'inflow_Aug': 'H06_I_Aug', 'inflow_Sep': 'H06_I_Sep', 'inflow_Oct': 'H06_I_Oct', 'inflow_Nov': 'H06_I_Nov', 'inflow

<xarray.Dataset>
Dimensions:                       (n: 3063)
Coordinates:
    ID                            (n) float64 ...
Dimensions without coordinates: n
Data variables: (12/61)
    HYPE_SPL_EL                   (n) float64 ...
    HYPE_SPL_COEFF                (n) float64 ...
    HYPE_SPL_EXP                  (n) float64 ...
    HYPE_LIM_EL                   (n) float64 ...
    HYPE_MIN_EL                   (n) float64 ...
    HYPE_ZERO_EL                  (n) float64 ...
    ...                            ...
    slope                         (n) float64 ...
    is_lake                       (n) float64 ...
    lake_vol                      (n) float64 ...
    lake_type_Doll                (n) float64 ...
    lake_type_Doll_HYPE           (n) float64 ...
    lake_type_Doll_HYPE_HANASAKI  (n) float64 ...
Attributes:
    Conventions:  CF-1.6
    License:      The data were written by Shervan Gharari. They are under GPL.
    history:      Created Tue Aug 22 11:17:43 2023
    source:       Written by test script of utilities (https://github.com/She...

In [ ]:
<varname_segId>                   ID
<varname_downSegId>               next_ID
<varname_hruArea>                 area
<varname_length>                  length
<varname_islake>                  is_lake
<varname_slope>                   slope
<varname_lakeModelType>           lake_type_Doll ! if all reservoirs are natural
<varname_lakeModelType>           lake_type_Doll_HYPE ! incusing HYPE and HANASAKI
<varname_lakeModelType>           lake_type_Doll_HYPE_HANASAKI ! incusing HYPE and HANASAKI
<varname_D03_Coefficient>         Doll_Coef
<varname_D03_Power>               Doll_Power
<varname_D03_MaxStorage>          lake_vol
<varname_D03_S0>                  Doll_S0
<varname_H06_alpha>               H06_alpha
<varname_H06_envfact>             H06_envfact
<varname_H06_c1>                  H06_c1
<varname_H06_c2>                  H06_c2
<varname_H06_exponent>            H06_exponent
<varname_H06_denominator>         H06_denominator
<varname_H06_c_compare>           H06_c_compare
<varname_H06_frac_Sdead>          H06_frac_Sdead
<varname_H06_I_mem_F>             H06_I_mem_F
<varname_H06_D_mem_F>             H06_D_mem_F
<varname_H06_I_mem_L>             H06_I_mem_L
<varname_H06_D_mem_L>             H06_D_mem_L
<varname_H06_Smax>                lake_vol
<varname_H06_S_ini>               lake_vol
<varname_H06_E_rel_ini>           H06_E_rel
<varname_H06_purpose>             H06_purpose
<varname_H06_I_Jan>               inflow_Jan
<varname_H06_I_Feb>               inflow_Feb
<varname_H06_I_Mar>               inflow_Mar
<varname_H06_I_Apr>               inflow_Apr
<varname_H06_I_May>               inflow_May
<varname_H06_I_Jun>               inflow_Jun
<varname_H06_I_Jul>               inflow_Jul
<varname_H06_I_Aug>               inflow_Aug
<varname_H06_I_Sep>               inflow_Sep
<varname_H06_I_Oct>               inflow_Oct
<varname_H06_I_Nov>               inflow_Nov
<varname_H06_I_Dec>               inflow_Dec
<varname_H06_D_Jan>               H06_D_Jan
<varname_H06_D_Feb>               H06_D_Feb
<varname_H06_D_Mar>               H06_D_Mar
<varname_H06_D_Apr>               H06_D_Apr
<varname_H06_D_May>               H06_D_May
<varname_H06_D_Jun>               H06_D_Jun
<varname_H06_D_Jul>               H06_D_Jul
<varname_H06_D_Aug>               H06_D_Aug
<varname_H06_D_Sep>               H06_D_Sep
<varname_H06_D_Oct>               H06_D_Oct
<varname_H06_D_Nov>               H06_D_Nov
<varname_H06_D_Dec>               H06_D_Dec
<varname_HYP_E_emr>               EL_SPL
<varname_HYP_E_lim>               EL_LIM
<varname_HYP_E_min>               EL_MIN
<varname_HYP_E_zero>              EL_ZERO
<varname_HYP_Qrate_amp>           OPS_AMP
<varname_HYP_Qrate_phs>           OPS_PHA
<varname_HYP_Qrate_prim>          OPS_MEAN
<varname_HYP_Qrate_emr>           SPL_COEFF
<varname_HYP_Erate_emr>           SPL_EXP
<varname_HYP_A_avg>               area
<varname_HYP_prim_F>              PRM_FLAG






In [17]:
import xarray as xr
import numpy as np

# Assuming you have your xarray data in 'data_array' variable
# and 'subset_of_ids' is a list containing the IDs in the desired order

# Create a sample xarray dataset for illustration purposes
# Replace this with your actual data
data_array = xr.Dataset(
    {
        'ID': (['n'], [101, 102, 103, 104]),
        'variable1': (['n', 'm'], [[1, 2], [3, 4], [5, 6], [7, 8]]),
        'variable2': (['n', 'time'], [[9, 10, 11], [12, 13, 14], [15, 16, 17], [18, 19, 20]]),
    }
)

subset_of_ids = [104, 102, 101]  # Replace this with your subset of IDs

# Find the new order of the 'n' dimension based on the subset_of_ids
sorted_indices = np.array([data_array['ID'].values.tolist().index(id) for id in subset_of_ids])

print(sorted_indices)


# Reorder the xarray object based on the new_order
reordered_data_array = data_array.isel(n=sorted_indices)

reordered_data_array



[3 1 0]


<xarray.Dataset>
Dimensions:    (n: 3, m: 2, time: 3)
Dimensions without coordinates: n, m, time
Data variables:
    ID         (n) int64 104 102 101
    variable1  (n, m) int64 7 8 3 4 1 2
    variable2  (n, time) int64 18 19 20 12 13 14 9 10 11

In [11]:
data_array

<xarray.Dataset>
Dimensions:    (n: 4, m: 2, time: 3)
Dimensions without coordinates: n, m, time
Data variables:
    ID         (n) int64 101 102 103 104
    variable1  (n, m) int64 1 2 3 4 5 6 7 8
    variable2  (n, time) int64 9 10 11 12 13 14 15 16 17 18 19 20

In [12]:
reordered_data_array

<xarray.Dataset>
Dimensions:    (n: 4, m: 2, time: 3)
Dimensions without coordinates: n, m, time
Data variables:
    ID         (n) int64 103 102 104 101
    variable1  (n, m) int64 5 6 3 4 7 8 1 2
    variable2  (n, time) int64 15 16 17 12 13 14 18 19 20 9 10 11